In [11]:
from langchain_groq import ChatGroq


In [13]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [18]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/?jobSearch=true&jsOffset=0&jsSort=posting_start_date&jsLanguage=en")
page_data = loader.load().pop().page_content
print(page_data)


Nike Careers Home

Section 1: Move the World. Welcome to Nike Careers.Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsMove the WorldWelcome to Nike CareersSearch JobsScrollSearch JobsFin

In [19]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)


str

In [20]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res


[{'role': 'Administrative Support',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Advanced Innovation',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Air Manufacturing Innovation',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Aviation', 'experience': None, 'skills': None, 'description': None},
 {'role': 'Communications',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Customer Service',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Design', 'experience': None, 'skills': None, 'description': None},
 {'role': 'Digital', 'experience': None, 'skills': None, 'description': None},
 {'role': 'Facilities',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Finance & Accounting',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Government & Public Affairs',
  'experience': None,
  'skills': None,
  'desc

In [21]:
type(json_res)

list

In [22]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [25]:
import uuid
import chromadb
import pandas as pd

# Initialize PersistentClient
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

# Sample DataFrame (replace with your actual DataFrame)
data = {'Techstack': ['Python', 'React', 'SQL'], 'Links': ['link1', 'link2', 'link3']}
df = pd.DataFrame(data)

# Populate collection if empty
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=[row["Techstack"]],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

# Define 'job' with 'skills'
job = {'skills': 'Python, React'}

# Query the collection
links = collection.query(query_texts=[job['skills']], n_results=2).get('metadatas', [])
print(links)


[[{'links': 'https://example.com/react-portfolio'}, {'links': 'https://example.com/react-native-portfolio'}]]


In [26]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links


[[{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [27]:
job


{'skills': 'Python, React'}

In [29]:
import json

# Simulated JSON response (Replace this with the actual response from your API or source)
json_res = [
    {"name": "Software Developer", "skills": "Python, React, SQL"},
    {"name": "Backend Engineer", "skills": "Node.js, Docker, Kubernetes"}
]

# Ensure the response is a list of dictionaries
if isinstance(json_res, list) and all(isinstance(item, dict) for item in json_res):
    # Access the first job
    job = json_res[0]  # Change the index to access different items
    if 'skills' in job:
        print(job['skills'])  # Output: Python, React, SQL
    else:
        print("Key 'skills' not found in the first job.")
else:
    print("Invalid structure in json_res.")
json_res = json.loads(json_string)


Python, React, SQL


In [31]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Alex, a business development executive at Nvidia. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Expert Software Development Solutions for Nvidia

Dear Alex,

I came across the job description for a Software Developer at Nvidia, requiring expertise in Python, React, and SQL. I'd like to introduce AtliQ, a leading AI & Software Consulting company that can fulfill your software development needs.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions, driving scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts can design and develop customized software applications that meet your specific requirements.

Our portfolio showcases our capabilities in React development, with notable projects featured on our website: https://example.com/react-portfolio. We have also successfully delivered React Native projects, which can be viewed at https://example.com/react-native-portfolio.

By partnering with AtliQ, you can leverage our technical expertise to drive innovation and growth at Nvidia. I'd be